In [2]:
import numpy as np
import re
import time
import pandas as pd

# spatial imports
import geopandas as gpd
import pygeos

# FCA specific imports
from access import Access, weights

/Users/hanna/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [20]:
supply = pd.read_csv("engin_rel_ct_supply.csv") # engin_rel_tr_supply, engin_ct_supply, ...
demand = pd.read_csv("prog_ct_demand.csv") # prog_ct_demand.csv
times = pd.read_csv("engin_rel_ct_drive_times.csv") # engin_rel_re_drive_times.csv. engin_ct_drive_times.csv ...
times = times.dropna()
demand = demand.dropna()
supply = supply.dropna()


In [21]:
# demand

,GEOID,NAME,popE,popM,geometry,lon,lat,geometry.1
0,51041100206,"Census Tract 1002.06, Chesterfield County, Vir...",5218,488,"c(-77.5334912927496, 37.4547728146469)",-77.533491,37.454773,"c(-77.5334912927496, 37.4547728146469)"
1,51059481202,"Census Tract 4812.02, Fairfax County, Virginia",4632,340,"c(-77.370777247977, 38.9425931851988)",-77.370777,38.942593,"c(-77.370777247977, 38.9425931851988)"
2,51710004800,"Census Tract 48, Norfolk city, Virginia",744,80,"c(-76.2799115849972, 36.8446379977949)",-76.279912,36.844638,"c(-76.2799115849972, 36.8446379977949)"
3,51117930300,"Census Tract 9303, Mecklenburg County, Virginia",3037,274,"c(-78.2500399870202, 36.7273772740616)",-78.250040,36.727377,"c(-78.2500399870202, 36.7273772740616)"
4,51143010300,"Census Tract 103, Pittsylvania County, Virginia",3288,313,"c(-79.4799204374501, 36.9886828312315)",-79.479920,36.988683,"c(-79.4799204374501, 36.9886828312315)"
...,...,...,...,...,...,...,...,...
1895,51590000300,"Census Tract 3, Danville city, Virginia",2562,297,"c(-79.3911488979682, 36.6044700154587)",-79.391149,36.604470,"c(-79.3911488979682, 36.6044700154587)"
1896,51197050200,"Census Tract 502, Wythe County, Virginia",5671,383,"c(-81.2212513540019, 36.972624986127)",-81.221251,36.972625,"c(-81.2212513540019, 36.972624986127)"
1897,51700031602,"Census Tract 316.02, Newport News city, Virginia",3484,297,"c(-76.4668048518816, 37.0701152891273)",-76.466805,37.070115,"c(-76.4668048518816, 37.0701152891273)"
1898,51059482202,"Census Tract 4822.02, Fairfax County, Virginia",3586,316,"c(-77.3460273941124, 38.9569849832375)",-77.346027,38.956985,"c(-77.3460273941124, 38.9569849832375)"


In [22]:
# load in times matrix - converting columns to integers
times.origin = times.origin.astype(int)
times.dest = times.dest.astype(int)

# load in supply matrix - converting columns to integers
supply.GEOID = supply.GEOID.astype(int)

# load in demand matrix - converting columns to integers
demand.GEOID = demand.GEOID.astype(int)


In [23]:
# weight for enchanced fca
fn30 = weights.step_fn({10:1, 20:0.68, 30:0.22})
fn30

<function access.weights.step_fn.<locals>.helper(key_to_test)>

In [24]:
# create Access object
A = Access(demand_df = demand,
           demand_index = "GEOID",
           #demand_index = "health_district",
           demand_value = "popE",
           supply_df = supply,
           supply_index = "GEOID",
           #supply_index = "health_district",
           supply_value = "capacity",
           cost_df = times,
           cost_origin = "origin",
           cost_dest = "dest",
           cost_name = "cost",
            neighbor_cost_df = times,
           neighbor_cost_origin = "origin",
           neighbor_cost_dest = "dest",
           neighbor_cost_name = "cost")
A

In [25]:
# obtain fcas
info_fca5 = A.fca_ratio(name = "fca5", max_cost = 5)
info_fca10 = A.fca_ratio(name = "fca10", max_cost = 10)
info_fca15 = A.fca_ratio(name = "fca15", max_cost = 15)
info_2sfca5 = A.two_stage_fca(name = "2sfca5", max_cost = 5)
info_2sfca10 = A.two_stage_fca(name = "2sfca10", max_cost = 10)
info_2sfca15 = A.two_stage_fca(name = "2sfca15", max_cost = 15)


/Users/hanna/opt/anaconda3/lib/python3.8/site-packages/access/fca.py:137: UserWarning: some tracts may be unaccounted for in supply_cost
  warnings.warn("some tracts may be unaccounted for in supply_cost", stacklevel = 1)
/Users/hanna/opt/anaconda3/lib/python3.8/site-packages/access/fca.py:137: UserWarning: some tracts may be unaccounted for in supply_cost
  warnings.warn("some tracts may be unaccounted for in supply_cost", stacklevel = 1)
/Users/hanna/opt/anaconda3/lib/python3.8/site-packages/access/fca.py:137: UserWarning: some tracts may be unaccounted for in supply_cost
  warnings.warn("some tracts may be unaccounted for in supply_cost", stacklevel = 1)


In [26]:
enh_2sfca = A.enhanced_two_stage_fca(name = "2sfca30", weight_fn = fn30)
info_3sfca = A.three_stage_fca(name = "3sfca", normalize=True)
df = pd.concat([info_3sfca, enh_2sfca], axis=1)
df.describe()

,3sfca_capacity,2sfca30_capacity
count,1899.000000,1899.000000
mean,1.105710,0.000578
std,1.711374,0.001002
min,0.000000,0.000000
25%,0.116916,0.000000
50%,0.648055,0.000260
75%,1.309708,0.000675
max,14.410786,0.008246


In [27]:
df.to_csv("/Users/hanna/Documents/SDAD Files/VDH Access to Quality Education/programs_data/3sfca_engin_rel_ct.csv")

In [34]:
# full access matrix, doesn't allow enhanced weights 
# access_df = A.norm_access_df
#access_df.to_csv("/Users/hanna/Documents/SDAD Files/VDH Access to Quality Education/college_datasets/access_less_two_tr.csv")

In [42]:
#access_df["3sfca_capacity"].describe()

count    1899.000000
mean        1.045904
std         1.139574
min         0.000000
25%         0.066759
50%         0.834350
75%         1.555948
max         4.729496
Name: 3sfca_capacity, dtype: float64